In [83]:
# TẢI DỮ LIỆU VÀ CÁC PACKAGE CẦN THIẾT  

import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
import seaborn as sns 
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 
pd.set_option('display.float_format', '{:.4f}'.format)

file_path = "../data/imputed_data.xlsx"
xls = pd.ExcelFile(file_path)
df = xls.parse(sheet_name="Sheet1")

# display(input_df.head(5))
# display(df.shape) # có 341 hàng và 38 cột
# display(df.columns) 

In [84]:
# TẠO HAI CỘT HOMA_IR ĐỂ ĐÁNH GIÁ ĐỀ KHÁNG INSULIN

df["homa_ir"] = (df["insulin"] * df["glucose_ac"]) / 405

df["insulin_re"] = np.where(df["homa_ir"] <= 3.16, 0, 1)

In [ ]:
# NHÃN BIẾN DANH ĐỊNH 

cpp_code = {0: "no", 1: "yes"}
sex_code = {0: "female", 1: "male"}
pregnancy_smoking_code = {0: "no", 1: "yes"}
# GDM_code = {"no-dia": 1, "gmd": 2, "other-dia": 3}
GDM_code = {1: "no", 2: "yes", 3: "no"}
preterm_birth_code = {0: "no", 1: "yes"}
father_diabetes_code = {0: "no", 1: "yes"}
mother_diabetes_code = {0: "no", 1: "yes"}
# education_level_code = {"senior high": 1, "vocational": 2, "graduate": 3, "college": 4}
education_level_code = {1: "low", 2: "low", 3: "high", 4: "medium"}
insulin_re_code = {0: "no", 1: "yes"}

In [86]:
# BIẾN ĐỔI NHÃN BIẾN DANH ĐỊNH

df["CPP"] = df["CPP"].map(cpp_code).astype("category")
df["sex"] = df["sex"].map(sex_code).astype("category")
df["pregnancy_smoking"] = df["pregnancy_smoking"].map(pregnancy_smoking_code).astype("category")
df["GDM"] = df["GDM"].map(GDM_code).astype("category")
df["preterm_birth"] = df["preterm_birth"].map(preterm_birth_code).astype("category")
df["father_diabetes"] = df["father_diabetes"].map(father_diabetes_code).astype("category")
df["mother_diabetes"] = df["mother_diabetes"].map(mother_diabetes_code).astype("category")

custom_categories = ["low", "medium", "high"]
df["education_level"] = pd.Categorical(
    df["education_level"].map(education_level_code),
    categories=custom_categories,
    ordered=True
)

df["insulin_re"] = df["insulin_re"].map(insulin_re_code).astype("category")

In [87]:
# TẠO MỘT SỐ CỘT MỚI 

# lbw
df["low_birth_weight"] = np.where(
    df["birth_weight_gram"] < 2500,
    "yes",
    "no"
)
df["low_birth_weight"] = df["low_birth_weight"].astype("category")

# preterm
df["preterm"] = np.where(
    df["gestational_age_week"] < 37,
    "yes",
    "no"
) # cái này đáng tin hơn, dường như cột preterm_birth khi trẻ sinh < 36 tuần
df["preterm"] = df["preterm"].astype("category")

# exclusive breast feeding time
df["breast_feeding_above_six"] = np.where(
    df["exclusive_breastfeeding_month"] >= 6,
    "yes",
    "no"
)
df["breast_feeding_above_six"] = df["breast_feeding_above_six"].astype("category")

# total breast feeding time
df["total_breastfeeding_month"] = df["exclusive_breastfeeding_month"] + df["mixed_breastfeeding_month"]
df["total_breastfeeding_above_six"] = np.where(
    df["total_breastfeeding_month"] >= 6,
    "yes",
    "no"
)
df["total_breastfeeding_above_six"] = df["total_breastfeeding_above_six"].astype("category")

# obesity or not
def determine_obesity(row):
    if pd.isna(row["zbmi"]) or pd.isna(row["age"]):
        return np.nan  # tránh lỗi khi dữ liệu thiếu
    if row["age"] < 5:
        return "yes" if row["zbmi"] > 3 else "no"
    else:
        return "yes" if row["zbmi"] > 2 else "no"
df["obesity"] = df.apply(determine_obesity, axis=1)
df["obesity"] = df["obesity"].astype("category")

# display(df["low_birth_weight"].value_counts())
# display(df["preterm"].value_counts())
# display(df["breast_feeding_above_six"].value_counts())
# display(df["total_breastfeeding_above_six"].value_counts())
# display(df["obesity"].value_counts())

In [88]:
# MỨC ĐỘ TĂNG CÂN TRONG THAI KÌ

# 1. Tạo DataFrame dữ liệu tham chiếu (GWG_ref)
GWG_ref = pd.DataFrame({
    "gestational_age_week": list(range(14, 41)),
    "p25": [-1.07, -0.45, 0.13, 0.67, 1.19, 1.69, 2.17, 2.64, 3.10, 3.54, 3.98, 4.42, 4.85, 5.28, 5.71, 6.14, 6.56, 6.99, 7.41, 7.84, 8.27, 8.70, 9.14, 9.57, 10.01, 10.45, 10.89],
    "p75": [0.65, 1.32, 1.99, 2.64, 3.29, 3.92, 4.55, 5.17, 5.79, 6.41, 7.02, 7.63, 8.24, 8.85, 9.45, 10.06, 10.67, 11.28, 11.89, 12.51, 13.12, 13.74, 14.37, 14.99, 15.62, 16.25, 16.89]
})

# 2. Tạo DataFrame dữ liệu mẫu (Có thêm dữ liệu thiếu)
target_df = df[["gestational_age_week", "gestational_weight_gain"]]

# 3. Nối (Merge) dữ liệu của bạn với bảng tham chiếu
df_merged = pd.merge(target_df, GWG_ref, on='gestational_age_week', how='left')

# 4. Phân loại mức tăng cân GWG (CÓ XỬ LÝ MISSING DATA)
# Kiểm tra dữ liệu thiếu trong 3 cột cần thiết cho việc phân loại:
# 1. Tuần tuổi thai (để merge thành công)
# 2. Tăng cân thực tế
# 3. p25/p75 (để đảm bảo tuần tuổi thai nằm trong khoảng tham chiếu)
missing_data_condition = (
    df_merged['gestational_age_week'].isna() |  # Thiếu tuần tuổi thai
    df_merged['gestational_weight_gain'].isna() | # Thiếu mức tăng cân
    df_merged['p25'].isna() # Tuần tuổi thai ngoài khoảng 14-40
)

conditions = [
    missing_data_condition,                                       # Điều kiện 1: Dữ liệu bị thiếu
    (df_merged['gestational_weight_gain'] < df_merged['p25']),     # Điều kiện 2: Dưới p25
    (df_merged['gestational_weight_gain'] > df_merged['p75']),     # Điều kiện 3: Trên p75
]

choices = [
    "Không phân loại", # Kết quả cho Điều kiện 1
    "Thiếu",                               # Kết quả cho Điều kiện 2
    "Thừa",                              # Kết quả cho Điều kiện 3
]

# Áp dụng các điều kiện. 'Phù hợp' là mặc định (p25 <= GWG <= p75)
df['gwg_category'] = np.select(conditions, choices, default='Phù hợp')
df['gwg_category'] = df['gwg_category'].replace("Không phân loại", np.nan)
df['gwg_category'] = pd.Categorical(
    df['gwg_category'], 
    categories=['Phù hợp', "Thiếu", "Thừa"], 
    ordered=False
)



In [89]:
# KIỂM TRA LẠI CÁC BIẾN CATEGORICAL

cat_vars = [
    'sex',
    'pregnancy_smoking',
    'GDM',
    'preterm_birth',
    'father_diabetes',
    'mother_diabetes',
    'education_level', 
    'CPP', 
    'insulin_re', 
    'low_birth_weight', 
    'preterm', 
    'breast_feeding_above_six', 
    'total_breastfeeding_above_six', 
    'obesity', 
    'gwg_category'
] # có 15 biến categorical

import pandas as pd
import numpy as np

# Danh sách các biến phân loại của bạn
cat_vars = [
    'sex', 'pregnancy_smoking', 'GDM', 'preterm_birth', 'father_diabetes',
    'mother_diabetes', 'education_level', 'CPP', 'insulin_re', 
    'low_birth_weight', 'preterm', 'breast_feeding_above_six', 
    'total_breastfeeding_above_six', 'obesity', 'gwg_category'
]

print("--- KIỂM TRA CẤP ĐỘ THAM CHIẾU (REFERENCE LEVEL) ---")

for var in cat_vars:
    # 1. Đảm bảo biến là kiểu Categorical và không có giá trị thiếu (NaN)
    if var not in df.columns or df[var].dtype.name != 'category':
        print(f"LỖI: Biến '{var}' không phải là kiểu Categorical hoặc không tồn tại.")
        continue
        
    # Lấy danh sách các cấp độ (categories)
    categories = df[var].cat.categories
    
    # Nhãn đầu tiên luôn là Cấp độ Tham chiếu (Reference Level = 0)
    reference_level = categories[0] 
    
    # Các nhãn còn lại là cấp độ quan tâm (Interest Levels = 1)
    interest_levels = categories[1:]
    
    # Hiển thị kết quả
    print(f"\nBIẾN: {var}")
    print(f"  > Tất cả Nhãn: {list(categories)}")
    print(f"  > CẤP ĐỘ THAM CHIẾU (Mẫu số/0): **{reference_level}**")
    print(f"  > CẤP ĐỘ QUAN TÂM (Tử số/1): {list(interest_levels)}")

--- KIỂM TRA CẤP ĐỘ THAM CHIẾU (REFERENCE LEVEL) ---

BIẾN: sex
  > Tất cả Nhãn: ['female', 'male']
  > CẤP ĐỘ THAM CHIẾU (Mẫu số/0): **female**
  > CẤP ĐỘ QUAN TÂM (Tử số/1): ['male']

BIẾN: pregnancy_smoking
  > Tất cả Nhãn: ['no', 'yes']
  > CẤP ĐỘ THAM CHIẾU (Mẫu số/0): **no**
  > CẤP ĐỘ QUAN TÂM (Tử số/1): ['yes']

BIẾN: GDM
  > Tất cả Nhãn: ['no', 'yes']
  > CẤP ĐỘ THAM CHIẾU (Mẫu số/0): **no**
  > CẤP ĐỘ QUAN TÂM (Tử số/1): ['yes']

BIẾN: preterm_birth
  > Tất cả Nhãn: ['no', 'yes']
  > CẤP ĐỘ THAM CHIẾU (Mẫu số/0): **no**
  > CẤP ĐỘ QUAN TÂM (Tử số/1): ['yes']

BIẾN: father_diabetes
  > Tất cả Nhãn: ['no', 'yes']
  > CẤP ĐỘ THAM CHIẾU (Mẫu số/0): **no**
  > CẤP ĐỘ QUAN TÂM (Tử số/1): ['yes']

BIẾN: mother_diabetes
  > Tất cả Nhãn: ['no', 'yes']
  > CẤP ĐỘ THAM CHIẾU (Mẫu số/0): **no**
  > CẤP ĐỘ QUAN TÂM (Tử số/1): ['yes']

BIẾN: education_level
  > Tất cả Nhãn: ['low', 'medium', 'high']
  > CẤP ĐỘ THAM CHIẾU (Mẫu số/0): **low**
  > CẤP ĐỘ QUAN TÂM (Tử số/1): ['medium', 'high']

In [90]:
# KIỂM TRA CÁC BIẾN NUMERIC 

numeric_vars = df.select_dtypes(["int", "float"]).columns.to_list()

# len(numeric_vars) # có 32 biến số học 

numeric_vars

['age',
 'zbmi',
 'insulin',
 'glucose_ac',
 'cholesterol',
 'TG',
 'HDL',
 'cortisol',
 'waist',
 'hip',
 'SBP',
 'DBP',
 'family_income',
 'gestational_weight_gain',
 'gestational_age_week',
 'birth_weight_gram',
 'exclusive_breastfeeding_month',
 'mixed_breastfeeding_month',
 'father_BMI',
 'mother_BMI',
 'sedentary_lifestyle_hour_day',
 'low_physical_activity_hour_day',
 'food_weight(g)',
 'calories(kcal)',
 'fat(g)',
 'protein(g)',
 'total_carbohydrates(g)',
 'score_5A',
 'score_5B',
 'score_5C',
 'homa_ir',
 'total_breastfeeding_month']

In [91]:
# QUY ĐỊNH CÁC BIẾN SỐ CHO MÔ HÌNH

outcome_var = df["insulin_re"]

risk_factors = [
    # các confounder
    "sex", 
    "age", 
    "education_level", # cách tính trong nghiên cứu là rất khác, nó là biến số học, cộng điểm của cả cha và mẹ
    "zbmi",
    # các yếu tố nguy cơ prenatal
    "gestational_weight_gain", 
    "pregnancy_smoking", 
    "GDM", 
    # các yếu tố nguy cơ perinatal
    "birth_weight_gram", 
    "preterm_birth", # lưu ý ngưỡng chọn preterm là 36 tuần
    "gestational_age_week", 
    "exclusive_breastfeeding_month", 
    "mixed_breastfeeding_month"
]


In [ ]:
# CHẠY MÔ HÌNH LOGIT

# import pandas as pd
# import numpy as np
# import statsmodels.formula.api as smf


# df["insulin_re_bin"] = df["insulin_re"].map({"no": 0, "yes": 1}).astype("int") # Biến phụ thuộc (0 hoặc 1)
# # khi chuyển từ dạng dạng chữ sang số 0,1 nhớ ép kiểu dữ liệu thành int

# formula = "insulin_re_bin ~ " + " + ".join(risk_factors)
# model = smf.logit(formula, data=df)
# results = model.fit()

# print(results.summary())

# không hiểu vì lí do gì mà nếu dùng Logit đơn thuần (MLE method) thì không hội tụ, dù vẫn báo cáo kết quả
# ChatGPT bảo do cỡ mẫu quá nhỏ


In [ ]:
# CHẠY MÔ HÌNH GLM

import statsmodels.api as sm
import statsmodels.formula.api as smf


df["insulin_re_bin"] = df["insulin_re"].map({"no": 0, "yes": 1}).astype("int") # Biến phụ thuộc (0 hoặc 1)
# khi chuyển từ dạng dạng chữ sang số 0,1 nhớ ép kiểu dữ liệu thành int

formula = "insulin_re_bin ~ " + " + ".join(risk_factors)
# model = smf.logit(formula, data=df)
model = smf.glm(formula=formula, data=df, family=sm.families.Binomial())
results = model.fit()

print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:         insulin_re_bin   No. Observations:                  341
Model:                            GLM   Df Residuals:                      327
Model Family:                Binomial   Df Model:                           13
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -176.47
Date:                Thu, 23 Oct 2025   Deviance:                       352.94
Time:                        11:07:37   Pearson chi2:                     332.
No. Iterations:                    19   Pseudo R-squ. (CS):             0.2129
Covariance Type:            nonrobust                                         
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

In [ ]:
# TRÍCH XUẤT OR VÀ CI

params = results.params

conf = results.conf_int()

or_ci = pd.DataFrame({
    "params": params,
    "OR": np.exp(params),
    "2.5%": np.exp(conf[0]),
    "97.5%": np.exp(conf[1]),
    "p-value": results.pvalues.round(3)
})

display(or_ci)




,0,1
Intercept,-19.6944,9.0884
sex[T.male],-1.2393,0.0232
education_level[T.medium],-0.5378,0.7906
education_level[T.high],-0.5468,0.8053
pregnancy_smoking[T.yes],-24243.8952,24205.6856
GDM[T.yes],0.6040,4.3667
preterm_birth[T.yes],-0.9693,2.1876
age,0.0822,0.4107
zbmi,0.5298,0.9606
gestational_weight_gain,-0.0475,0.0543


In [100]:
# THỬ VỚI CÁC LOẠI BIẾN KHÁC

import statsmodels.api as sm
import statsmodels.formula.api as smf

outcome_var = df["insulin_re"]

risk_factors = [
    # các confounder
    "sex", 
    "age", 
    "education_level", # cách tính trong nghiên cứu là rất khác, nó là biến số học, cộng điểm của cả cha và mẹ
    "zbmi",
    # các yếu tố nguy cơ prenatal
    "gwg_category", 
    "GDM", 
    # các yếu tố nguy cơ perinatal
    "low_birth_weight", 
    "preterm", # lưu ý ngưỡng chọn preterm là 36 tuần
    "gestational_age_week", 
    "breast_feeding_above_six", 
    "total_breastfeeding_above_six"
]

formula = "insulin_re_bin ~ " + " + ".join(risk_factors)
# model = smf.logit(formula, data=df)
model = smf.glm(formula=formula, data=df, family=sm.families.Binomial())
results = model.fit()

print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:         insulin_re_bin   No. Observations:                  341
Model:                            GLM   Df Residuals:                      327
Model Family:                Binomial   Df Model:                           13
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -176.72
Date:                Thu, 23 Oct 2025   Deviance:                       353.43
Time:                        11:18:07   Pearson chi2:                     330.
No. Iterations:                     5   Pseudo R-squ. (CS):             0.2117
Covariance Type:            nonrobust                                         
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [1]:
import numpy as np
import pandas as pd

# Tạo dữ liệu
np.random.seed(42)
n = 1000
age = np.random.normal(50, 10, n)          # tuổi trung bình 50
bp = np.random.normal(140, 15, n)          # huyết áp trung bình 140
sex = np.random.binomial(1, 0.5, n)        # 0 = nữ, 1 = nam

# Gán nhóm ngẫu nhiên
Z = np.random.binomial(1, 0.5, n)          # 0 = control, 1 = treatment

# Gom vào DataFrame
df = pd.DataFrame({"Z": Z, "age": age, "bp": bp, "sex": sex})

# So sánh trung bình đặc điểm ban đầu giữa hai nhóm
summary = df.groupby("Z")[["age", "bp", "sex"]].mean().rename(index={0: "Control", 1: "Treatment"})
summary


,age,bp,sex
Z,,,
Control,50.511257,140.209793,0.522244
Treatment,49.853004,141.975323,0.482402
